In [2]:
%pip install sympy

Defaulting to user installation because normal site-packages is not writeable
  Using cached sympy-1.10.1-py3-none-any.whl (6.4 MB)
  Using cached mpmath-1.2.1-py3-none-any.whl (532 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sympy import symbols, Eq
from sympy import * 

import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

In [17]:
files = 170

number_of_nodes = []

cfg_name = 'TS{}.cfg'
cfg_list = []
col_list_full = []
for i in range(0,files):
     cfg_list.append(open(cfg_name.format(i)))
     lines = cfg_list[i].readlines()
     number_of_nodes.append(int(lines[30][14]+lines[30][15]))
     col_list_single = []
     for j in range(number_of_nodes[i]):
          if j == 9: col_list_single.append(lines[31+j][3])
          else: col_list_single.append(lines[31+j][2])
     col_list_full.append(col_list_single)

topo_name = 'TS{}.topo'
topo_list = []
for i in range(0,files):
     # if i == 1 or i == 10 or i == 13 or i == 23 or i == 26: continue
     topo_list.append(pd.read_csv(topo_name.format(i), sep = '\t'))
     topo_list[i]

for j in range(len(topo_list)):

     topo = topo_list[j]
     nodes = [['A',0,0],['B',1,0],['C',2,0],['D',3,0],['E',4,0],['F',5,0],['G',6,0],['H',7,0],['I',8,0],['J',9,0]]
     nodes = nodes[:number_of_nodes[j]]
     node_to_index_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9}

     for i in range (len(topo)):
          nodes[node_to_index_dict[topo['Source'][i]]][2] += 1
          # nodes[node_to_index_dict[topo['Target'][i]]][2] += 1
          nodes[node_to_index_dict[topo['Source'][i]]].append(node_to_index_dict[topo['Target'][i]])
          # nodes[node_to_index_dict[topo['Target'][i]]].append(node_to_index_dict[topo['Source'][i]])

     edges = 0
     for node_info in nodes:
          for conn in node_info[3:]:
               edges += 1

     n_of_nodes = len(nodes)
     reactions = n_of_nodes*2  # gonna assume order of reactions- A production, B production... , A degradation, B degradation... etc.

     # stoichiometric matrix
     A = []
     for row in range(n_of_nodes):
          temp = []
          for ele in range(reactions):
               if (ele == row): temp.append(1)
               elif (ele == row+n_of_nodes): temp.append(-1)
               else: temp.append(0)
          A.append(temp)
     
     # influence matrix
     Z = []
     for row in range(reactions):
          temp = []
          for ele in range(n_of_nodes):
               temp.append(0)
          Z.append(temp)
     
     z = [None]*(edges+n_of_nodes)
     for i in range(edges+n_of_nodes):
          z[i] = symbols('z'+str(i))
     
     ind = 0
     for col in  range(n_of_nodes):
          for row in range(reactions):
               if (row>=n_of_nodes):
                    if (col == row-n_of_nodes): 
                         Z[row][col] = z[ind]
                         ind += 1
          for conn in nodes[col][3:]:
               Z[conn][col] = -z[ind]
               ind += 1

     A = Matrix(A)
     Z = Matrix(Z)
     A.shape
     F = Matrix(A.nullspace()).rref()
     F_rref = F[0].reshape(A.shape[0],A.shape[1])
     M = A*Z
     Mt = M
     s = A.rank()
     n = A.shape[0]
     # F_rref,Mt,s,n

     if s<n:
          i = 1
          # for i in range(A.shape[1]):
          while i<= A.shape[1]:
               j = 1
               while j<n+1:
                    if F_rref[(i-1)+(j-1)*n] != 0:
                         # row replacement
                         for k in range(Mt.shape[1]):
                              if k<F_rref.shape[1]: Mt[(j-1)*Mt.shape[1]+k] = F_rref[(i-1)*F_rref.shape[1]+k]
                              else: Mt[(j-1)*Mt.shape[1]+k] = 0
                         j = n+1
                    else:
                         j = j+1
          i+=1

     det = Mt.det()
     det = det.expand()
     injective = 0
     if det == 0: injective = 0
     else:
          all_pos = 1
          all_neg = 1
          for term in det.args:
               if term.args[0] != 1 and term.args[0] != -1: all_neg = 0
               elif term.args[0] > 0: all_neg = 0
               else: all_pos = 0
          if all_neg == 0 and all_pos == 0: injective = 0
          else: injective = 1
     if (injective == 1):
          print (topo)
          print (j)
          print(det)

In [13]:
type(det)

sympy.core.mul.Mul

In [4]:
pd.set_option('display.max_rows',1000)

In [15]:
det.expand()

-z0*z11*z12*z16*z17*z20*z5*z7 + z0*z11*z12*z16*z17*z21*z6*z7 + z0*z11*z13*z16*z19*z20*z5*z7 - z0*z11*z13*z16*z19*z21*z6*z7 + z0*z12*z14*z17*z20*z5*z7*z9 - z0*z12*z14*z17*z21*z6*z7*z9 - z0*z13*z14*z19*z20*z5*z7*z9 + z0*z13*z14*z19*z21*z6*z7*z9 + z1*z10*z12*z16*z17*z20*z5*z7 - z1*z10*z12*z16*z17*z21*z6*z7 - z1*z10*z13*z16*z19*z20*z5*z7 + z1*z10*z13*z16*z19*z21*z6*z7 - z1*z12*z15*z17*z20*z5*z7*z9 + z1*z12*z15*z17*z21*z6*z7*z9 + z1*z13*z15*z19*z20*z5*z7*z9 - z1*z13*z15*z19*z21*z6*z7*z9 - z10*z12*z14*z17*z20*z4*z5*z7 + z10*z12*z14*z17*z21*z4*z6*z7 + z10*z13*z14*z19*z20*z4*z5*z7 - z10*z13*z14*z19*z21*z4*z6*z7 + z11*z12*z15*z17*z20*z4*z5*z7 - z11*z12*z15*z17*z21*z4*z6*z7 + z11*z12*z16*z17*z20*z3*z5*z8 - z11*z12*z16*z17*z21*z3*z6*z8 + z11*z12*z16*z18*z2*z20*z5*z7 - z11*z12*z16*z18*z2*z21*z6*z7 - z11*z13*z15*z19*z20*z4*z5*z7 + z11*z13*z15*z19*z21*z4*z6*z7 - z11*z13*z16*z19*z20*z3*z5*z8 + z11*z13*z16*z19*z21*z3*z6*z8 - z12*z14*z17*z20*z3*z5*z8*z9 + z12*z14*z17*z21*z3*z6*z8*z9 - z12*z14*z18*z2*z2

In [ ]:
A.shape
F = Matrix(A.nullspace()).rref()
F_rref = F[0].reshape(A.shape[0],A.shape[1])
M = A*Z
Mt = M
s = A.rank()
n = A.shape[0]
F_rref,Mt,s,n

if s<n:
    i = 1
    # for i in range(A.shape[1]):
    while i<= A.shape[1]:
        j = 1
        while j<n+1:
            if F_rref[(i-1)+(j-1)*n] != 0:
                # row replacement
                for k in range(Mt.shape[1]):
                    if k<F_rref.shape[1]: Mt[(j-1)*Mt.shape[1]+k] = F_rref[(i-1)*F_rref.shape[1]+k]
                    else: Mt[(j-1)*Mt.shape[1]+k] = 0
                j = n+1
            else:
                j = j+1
    i+=1

det = Mt.det()

injective = 0
if det == 0: injective = 0
else:
    all_pos = 1
    all_neg = 1
    for term in det.args:
        if term.args[0] > 0: all_neg = 0
        else: all_pos = 0
    if all_neg == 0 and all_pos == 0: injective = 0
    else: injective = 1

In [ ]:
A.shape
F = Matrix(A.nullspace()).rref()
F_rref = F[0].reshape(A.shape[0],A.shape[1])
M = A*Z
Mt = M
s = A.rank()
n = A.shape[0]
F_rref,Mt,s,n

if s<n:
  i = 1
  # for i in range(A.shape[1]):
  while i<= A.shape[1]:
    j = 1
    while j<n+1:
      if F_rref[(i-1)+(j-1)*n] != 0:
        # row replacement
        for k in range(Mt.shape[1]):   
          if k<F_rref.shape[1]: Mt[(j-1)*Mt.shape[1]+k] = F_rref[(i-1)*F_rref.shape[1]+k]
          else: Mt[(j-1)*Mt.shape[1]+k] = 0
        j = n+1
      else:
        j = j+1
  i+=1

det = Mt.det()

injective = 0
if det == 0: injective = 0
else:
  all_pos = 1
  all_neg = 1
  for term in det.args:
    if term.args[0] > 0: all_neg = 0
    else: all_pos = 0
  if all_neg == 0 and all_pos == 0: injective = 0
  else: injective = 1

In [30]:
# ind = 0
# for col in  range(n_of_nodes):
#      for row in range(reactions):
#           if (row>=n_of_nodes):
#                if (col == row-n_of_nodes): 
#                     Z[row][col] = z[ind]
#                     ind += 1
#      for conn in nodes[col][3:]:
#           Z[conn][col] = -z[ind]
#           ind += 1

In [7]:

z = [None]*6
z[0] = symbols('z0')
z[1] = symbols('z1')
z[2] = symbols('z2')
z[3] = symbols('z3')
z[4] = symbols('z4')
z[5] = symbols('z5')

# stoichiometric matrix
A = Matrix([[-1,1,0,0,0,0],
     [0,0,-1,1,0,0],
     [0,0,0,0,-1,1]])

# influence matrix
Z = Matrix([[z[0],0,0],
     [0,0,-z[1]],
     [0,z[2],0],
     [-z[3],0,0],
     [0,0,z[4]],
     [0,-z[5],0]])

In [3]:
A.shape
F = Matrix(A.nullspace()).rref()
F_rref = F[0].reshape(A.shape[0],A.shape[1])
M = A*Z
Mt = M
s = A.rank()
n = A.shape[0]
F_rref,Mt,s,n

(Matrix([
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]),
 Matrix([
 [-z0,   0, -z1],
 [-z3, -z2,   0],
 [  0, -z5, -z4]]),
 3,
 3)

In [4]:
if s<n:
  i = 1
  # for i in range(A.shape[1]):
  while i<= A.shape[1]:
    j = 1
    while j<n+1:
      if F_rref[(i-1)+(j-1)*n] != 0:
        # row replacement
        for k in range(Mt.shape[1]):   
          if k<F_rref.shape[1]: Mt[(j-1)*Mt.shape[1]+k] = F_rref[(i-1)*F_rref.shape[1]+k]
          else: Mt[(j-1)*Mt.shape[1]+k] = 0
        j = n+1
      else:
        j = j+1
  i+=1



In [7]:
det = Mt.det()
print(Mt)

Matrix([[-z0, 0, -z1], [-z3, -z2, 0], [0, -z5, -z4]])


In [8]:
injective = 0
if det == 0: injective = 0
else:
  all_pos = 1
  all_neg = 1
  for term in det.args:
    if term.args[0] > 0: all_neg = 0
    else: all_pos = 0
  if all_neg == 0 and all_pos == 0: injective = 0
  else: injective = 1

In [9]:
A[0]

-1